## Datos Lenguas Andinas

In [1]:
## juntemos todo en una sola función :)
import pandas as pd

In [2]:
ms = pd.read_csv("marking_strategy.csv", sep=";")
cs = pd.read_csv("constructional_structure.csv", sep=";")
cs = cs.drop(columns = ['Andean'])

In [3]:
andes = pd.merge(ms,cs, on='Language')

In [4]:
andes = andes.to_dict(orient='split')
andes = andes['data']

In [5]:
andes = {lengua[1]:lengua[2:] for lengua in andes if lengua[0] == 1}

In [6]:
andes = {key.lower():value for key,value in andes.items()}

In [7]:
D

NameError: name 'D' is not defined

In [ ]:
andes

In [ ]:
len(andes)

### mapas

In [ ]:
import pandas as pd

lenguas_glotto = pd.read_csv('languoid.csv',sep=',',encoding='utf-8')


In [ ]:
lenguas_glotto[10:]

In [ ]:
changes = {'amar1274':'hara1260','trio1238':'kawe1237','alac1240':'tiri1260','wich1264':'wich1261'}
for code in changes.keys():
    lenguas_glotto.loc[(lenguas_glotto.id == code),'id']=changes[code]

In [ ]:
lenguas_glotto['ubicacion'] = lenguas_glotto['latitude'].astype(str) + '/' + lenguas_glotto['longitude'].astype(str)
lenguas_glotto = lenguas_glotto.drop(columns = ['latitude','longitude'])

In [ ]:
glotto_coordinates = dict(zip([name.lower() for name in lenguas_glotto['id']],lenguas_glotto['ubicacion']))
glotto_coordinates

In [ ]:
glotto_coordinates = {key:glotto_coordinates[key].split(' / ') for key in glotto_coordinates.keys()}

In [ ]:
glotto_coordinates['mill1237']=['-6.68', '-73.24']
glotto_coordinates['alle1238']=['3.11', '-66.24']

In [ ]:
lenguas_glotto = dict(zip([name.lower() for name in lenguas_glotto['name']],lenguas_glotto['id']))

In [ ]:
lenguas_glotto = {key:lenguas_glotto[key] for key in lenguas_glotto.keys() if key in andes.keys()}

In [ ]:
len(lenguas_glotto)

In [ ]:
faltan = [item for item in list(andes.keys()) if item not in list(lenguas_glotto.keys())]

In [ ]:
faltan

In [ ]:
glotto_lenguas = {v: k for k, v in lenguas_glotto.items()}

In [ ]:
## miramos languoids

languoid = pd.read_csv('Languages.csv',sep=',')

In [ ]:
languoid

In [ ]:
languoid = languoid[languoid['ID'].isin(lenguas_glotto.values())]

In [ ]:
languoid

In [ ]:
len(languoid)

In [ ]:
for i in languoid.index:
    if languoid.isnull().loc[i,'Latitude']:
        code=languoid.loc[i,'ID']
        print(code)
        if glotto_coordinates[code][0]!='/':
            languoid.at[i, 'Latitude'] = glotto_coordinates[code][0]
            languoid.at[i, 'Longitude'] = glotto_coordinates[code][1]

In [ ]:
languoid

In [ ]:
import geopandas

In [ ]:
import descartes

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

In [ ]:
world = world[world['continent']=='South America']

In [ ]:
x, y, codes = languoid['Longitude'].values, languoid['Latitude'].values, languoid['ID'].values

In [ ]:
len(x)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(dpi=800)

ax.set_aspect('equal')
plt.title('Languages of South America',fontsize=7)
world.plot(ax=ax, color='white', edgecolor='salmon',linewidth=0.2)
#cities.plot(ax=ax, marker='o', color='red', markersize=5)
ax.plot(x, y, marker='o',color='gold',linewidth=0,markersize=4,markeredgewidth=0.25,markeredgecolor='k',alpha=0.75)
annotate = True
if annotate == True:
    for i in range(len(x)):
        ax.annotate(codes[i], weight='demi', color='k', xy=(x[i],y[i]), fontsize=1.5, alpha=0.65)
ax.set_yticks([])
ax.set_xticks([])
plt.savefig('maps/mapa'+str(annotate)+'.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
Dcodes = {lenguas_glotto[key]:D[key] for key in D.keys()}

In [ ]:
rasgos = []

for key in Dcodes.keys():
    rasgos+=list(Dcodes[key].keys())
rasgos = list(set(rasgos))

In [ ]:
rasgos

In [ ]:
def values_per_feature(DD,f):
    DDD = {}
    for key in DD.keys():
        if f in DD[key].keys():
            DDD[key]=DD[key][f]
    return DDD

In [ ]:
values_per_feature(Dcodes,'4_4')

In [ ]:
import matplotlib.pyplot as plt

dict_color = {'1':{'marker':'X','color':'m'},'2':{'marker':'o','color':'b'},'3':{'marker':'s','color':'r'},
             '4':{'marker':'D','color':'cyan'},'5':{'marker':'*','color':'yellow'},'6':{'marker':'^','color':'fuchsia'},
             '7':{'marker':'v','color':'olive'},'8':{'marker':'p','color':'lime'},'9':{'marker':'H','color':'salmon'}}

for rasgo in rasgos:
    
    fig, ax = plt.subplots(dpi=800)

    ax.set_aspect('equal')
    plt.title('Languages for the negation domain: feature '+rasgo[2:],fontsize=5)
    world.plot(ax=ax, color='white', edgecolor='black',linewidth=0.2)
#cities.plot(ax=ax, marker='o', color='red', markersize=5)
#ax.scatter(x, y, marker="o", color='gold', alpha=1., zorder=5, s=10)
    for f in set(values_per_feature(Dcodes,rasgo).values()):
        
        ax.plot([x[i] for i in [list(codes).index(language) for language in codes if language in values_per_feature(Dcodes,rasgo).keys() and values_per_feature(Dcodes,rasgo)[language]==f]],[y[i] for i in [list(codes).index(language) for language in codes if language in values_per_feature(Dcodes,rasgo).keys() and values_per_feature(Dcodes,rasgo)[language]==f]],marker=dict_color[str(f)]['marker'],color=dict_color[str(f)]['color'],linewidth=0,markersize=5,markeredgewidth=0.5,markeredgecolor='k',alpha=0.75,fillstyle='full',clip_on=True,label='value '+str(f))
    plt.legend(loc='best',fontsize=5)

    ax.set_yticks([])
    ax.set_xticks([])
    plt.savefig('maps/mapa_rasgo'+rasgo +'.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
    plt.show()

### ¿Cómo se distribuyen las lenguas según el número de rasgos?

In [ ]:
from collections import Counter

def distribucion(D):
    D_rasgos = {lengua:{} for lengua in D.keys()}
    for lengua in D.keys():
        for lengualengua in D.keys():
            features=list(D.values())
            features=[len(item) for item in features]
    C=dict(Counter(features))
    keys = sorted(C.keys())
    DD = {}
    for key in keys:
        x = 0
        for keykey in C.keys():
            if keykey>=key:
                x+=C[keykey]
        DD[key]=x
    return DD

In [ ]:
distribucion(D)

## Representaciones

In [ ]:
## filtramos los datos leídos

def filter_rasgos_comun(D,num_rasgos):
    
    ## buscamos los rasgos en común
    rasgos_comunes = [list(item.keys()) for item in D.values() if len(item)>num_rasgos]
    ## intersectamos todos los conjuntos de rasgos sujetos a la condición len(item)>num_rasgos
    rasgos_comunes = set(rasgos_comunes[0]).intersection(*rasgos_comunes)
    ## restringimos D
    D = {lengua:{key:D[lengua][key] for key in rasgos_comunes} for lengua in D.keys() if len(D[lengua])>num_rasgos}
    return D

In [ ]:
D = read_lenguas_andinas('TabladeDatos.csv','andean languages.csv',4)

In [ ]:
D

In [ ]:
def binarizer(D):
    B = {lengua:{} for lengua in D.keys()}
    for lengua in D.keys():
        for key in D[lengua].keys():
            B[lengua][key+'_'+str(D[lengua][key])]=1
            
    ## recolectamos los keys :)
    
    keys = []
    for lengua in B.keys():
        for key in B[lengua].keys():
            if key not in keys:
                keys+=[key]
                
    C = {lengua:{} for lengua in D.keys()}
    for lengua in D.keys():
        for key in D[lengua].keys():
            C[lengua][key+'_'+str(D[lengua][key])]=1
            for keykey in keys:
                if keykey.startswith(key):
                    if key+'_'+str(D[lengua][key])!=keykey:
                        C[lengua][keykey]=0
    
    return C

In [ ]:
#D=binarizer(D)

In [ ]:
D['mapudungun']

In [ ]:
Ks = {'4_1':0,'4_2':0,'4_3':0,'4_4':0}

for key in Ks:
    for keykey in D['mapudungun'].keys():
        if keykey.startswith(key):
            Ks[key]+=1

In [ ]:
Ks

In [ ]:
len(D['mochica'])

## Distancias inter e intra área!

In [ ]:
areas = pd.read_csv('areas.csv',sep=',')
lenguas = [name.lower() for name in list(areas['lengua'])]
areas_dict = {}
for i in list(areas.index):
    if areas.loc[i,'lengua'].lower()=='moseten':
        areas_dict['moseten']=[areas.loc[i,'sub-area'].lower(),areas.loc[i,'area'].lower()]
    else:
        areas_dict[areas.loc[i,'lengua'].lower()]=[areas.loc[i,'sub-area'].lower().strip(),areas.loc[i,'area'].lower().strip()]

In [ ]:
areas_dict

In [ ]:
areas

In [ ]:
area_dict = {language:areas_dict[language][1] for language in areas_dict.keys()}
subarea_dict = {language:areas_dict[language][0] for language in areas_dict.keys()}

In [ ]:
import random
from collections import defaultdict

area_random = []

keys = list(area_dict.keys())
for i in range(1000):
    values = list(area_dict.values())
    values = random.sample(values,len(values))
    d=dict(zip(keys,values))
    
    v = defaultdict(list)
    for key, value in sorted(d.items()):
        v[value].append(key)
        
    area_random+=[{area:[l for l in dict(v)[area] if l in D.keys()] for area in dict(v).keys()}]

In [ ]:
subarea_random = []

keys = list(subarea_dict.keys())
for i in range(1000):
    values = list(subarea_dict.values())
    values = random.sample(values,len(values))
    d=dict(zip(keys,values))
    
    v = defaultdict(list)
    for key, value in sorted(d.items()):
        v[value].append(key)
        
    subarea_random+=[{area:[l for l in dict(v)[area] if l in D.keys()] for area in dict(v).keys()}]

In [ ]:
from collections import defaultdict

v = defaultdict(list)

for key, value in sorted(area_dict.items()):
    v[value].append(key)

group_area = {area:[l for l in dict(v)[area] if l in D.keys()] for area in dict(v).keys()}

In [ ]:
group_area

In [ ]:
v = defaultdict(list)

for key, value in sorted(subarea_dict.items()):
    v[value].append(key)

group_subarea = {area:[l for l in dict(v)[area] if l in D.keys()] for area in dict(v).keys()}

In [ ]:
for group in group_subarea.keys():
    print(group,len(group_subarea[group]))

In [ ]:
## distancia de Hamming 

def hamming(lengua1, lengua2):
    
    ## rasgos
    features_lengua1=D[lengua1]
    features_lengua2=D[lengua2]
    
    ## hamming!
    d=0
    n=0
    for feature in features_lengua1.keys():
        if feature in features_lengua2.keys(): 
            if features_lengua1[feature] != features_lengua2[feature]:
                d += 1.0
            n += 1.0
            
    return d/n

In [ ]:
D['mapudungun'],D['selknam']

In [ ]:
import itertools

def distancias_group(dict_group):
    D = {area:{} for area in dict_group.keys()}
    for area in dict_group.keys():
        for areaarea in dict_group.keys():
            lenguas = dict_group[area]
            lenguas_comp = dict_group[areaarea]
            L = [lenguas,lenguas_comp]
            pairs = list(itertools.product(*L))
            distances = 0
            for pair in pairs:
                distances+=hamming(pair[0],pair[1])
            distances = distances/len(pairs)
            D[area][areaarea]=distances

    return D

In [ ]:
distancias_area = distancias_group(group_area)

In [ ]:
distancias_area

In [ ]:
distancias_area_random = [distancias_group(d) for d in area_random]

In [ ]:
distancias_area_random[100]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

print(stats.ttest_1samp([d['chaco']['chaco'] for d in distancias_area_random], distancias_area['chaco']['chaco'],alternative="less"))

fig, ax = plt.subplots(dpi=800)
ax = sns.distplot([d['chaco']['chaco'] for d in distancias_area_random],kde_kws={"color": "k", "lw": 1, "label": "randomized Chaco"},
                  hist_kws={"histtype": "step", "linewidth": 1,
                            "alpha": 1, "color": "g"})

plt.axvline(x=distancias_area['chaco']['chaco'],linestyle='--',color='gold',label=r'Chaco')
plt.legend(loc='best',fontsize=7)
plt.xlabel(r'Hamming distance',fontsize=15)
plt.ylabel(r'frequency',fontsize=15)
plt.rcParams.update({'font.size': 10})
plt.savefig('chacovschaco.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

print(stats.ttest_1samp([d['andes']['andes'] for d in distancias_area_random], distancias_area['andes']['andes'],alternative="greater"))

fig, ax = plt.subplots(dpi=800)
ax = sns.distplot([d['andes']['andes'] for d in distancias_area_random],kde_kws={"color": "k", "lw": 1, "label": "randomized Andes"},
                  hist_kws={"histtype": "step", "linewidth": 1,
                            "alpha": 1, "color": "g"})

plt.axvline(x=distancias_area['andes']['andes'],linestyle='--',color='gold',label=r'Andes')
plt.legend(loc='best',fontsize=7)
plt.xlabel(r'Hamming distance',fontsize=15)
plt.ylabel(r'frequency',fontsize=15)
plt.rcParams.update({'font.size': 10})
plt.savefig('andesvsandes.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

print(stats.ttest_1samp([d['amazonia']['amazonia'] for d in distancias_area_random], distancias_area['amazonia']['amazonia'],alternative="greater"))

fig, ax = plt.subplots(dpi=800)
ax = sns.distplot([d['amazonia']['amazonia'] for d in distancias_area_random],kde_kws={"color": "k", "lw": 1, "label": "randomized Amazonia"},
                  hist_kws={"histtype": "step", "linewidth": 1,
                            "alpha": 1, "color": "g"})

plt.axvline(x=distancias_area['amazonia']['amazonia'],linestyle='--',color='gold',label=r'Amazonia')
plt.legend(loc='best',fontsize=7)
plt.xlabel(r'Hamming distance',fontsize=15)
plt.ylabel(r'frequency',fontsize=15)
plt.rcParams.update({'font.size': 10})
plt.savefig('amazoniavsamazonia.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
distancias_subarea = distancias_group(group_subarea)

In [ ]:
distancias_subarea

In [ ]:
distancias_subarea_random = [distancias_group(d) for d in subarea_random]

In [ ]:
distancias_subarea_random[100]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

print(stats.ttest_1samp([d['central andes']['central andes'] for d in distancias_subarea_random], distancias_subarea['central andes']['central andes'],alternative="greater"))

fig, ax = plt.subplots(dpi=800)
ax = sns.distplot([d['central andes']['central andes'] for d in distancias_subarea_random],kde_kws={"color": "k", "lw": 1, "label": "randomized Central Andes"},
                  hist_kws={"histtype": "step", "linewidth": 1,
                            "alpha": 1, "color": "g"})

plt.axvline(x=distancias_subarea['central andes']['central andes'],linestyle='--',color='gold',label=r'Central Andes')
plt.legend(loc='best',fontsize=7)
plt.xlabel(r'Hamming distance',fontsize=15)
plt.ylabel(r'frequency',fontsize=15)
plt.rcParams.update({'font.size': 10})
plt.savefig('centralandes.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

print(stats.ttest_1samp([d['central andes']['northern andes'] for d in distancias_subarea_random], distancias_subarea['central andes']['northern andes'],alternative="greater"))

fig, ax = plt.subplots(dpi=800)
ax = sns.distplot([d['central andes']['northern andes'] for d in distancias_subarea_random],kde_kws={"color": "k", "lw": 1, "label": "randomized Central vs. Northern Andes"},
                  hist_kws={"histtype": "step", "linewidth": 1,
                            "alpha": 1, "color": "g"})

plt.axvline(x=distancias_subarea['central andes']['northern andes'],linestyle='--',color='gold',label=r'Central vs. Northern Andes')
plt.legend(loc='best',fontsize=7)
plt.xlabel(r'Hamming distance',fontsize=15)
plt.ylabel(r'frequency',fontsize=15)
plt.rcParams.update({'font.size': 10})
plt.savefig('centralvsnorthernandes.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

print(stats.ttest_1samp([d['central andes']['southern andes'] for d in distancias_subarea_random], distancias_subarea['central andes']['southern andes'],alternative="greater"))

fig, ax = plt.subplots(dpi=800)
ax = sns.distplot([d['central andes']['southern andes'] for d in distancias_subarea_random],kde_kws={"color": "k", "lw": 1, "label": "randomized Central vs. Southern Andes"},
                  hist_kws={"histtype": "step", "linewidth": 1,
                            "alpha": 1, "color": "g"})

plt.axvline(x=distancias_subarea['central andes']['southern andes'],linestyle='--',color='gold',label=r'Central vs. Southern Andes')
plt.legend(loc='best',fontsize=7)
plt.xlabel(r'Hamming distance',fontsize=15)
plt.ylabel(r'frequency',fontsize=15)
plt.rcParams.update({'font.size': 10})
plt.savefig('centralvssouthernandes.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
D['allentiac']

In [ ]:
D['millcayac']

In [ ]:
D_lenguas = {lengua:{} for lengua in D.keys()}

for lengua in D.keys():
    for lengua_lengua in D.keys():
        lengua_feat = D[lengua]
        lengua_lengua_feat = D[lengua_lengua]
        D_lenguas[lengua][lengua_lengua]=hamming(lengua,lengua_lengua)

In [ ]:
D_andinas = {lengua:{} for lengua in group_area['andes']}

for lengua in group_area['andes']:
    for lengua_lengua in group_area['andes']:
        lengua_feat = D[lengua]
        lengua_lengua_feat = D[lengua_lengua]
        D_andinas[lengua][lengua_lengua]=hamming(lengua,lengua_lengua)

In [ ]:
D_andinas['mapudungun']

In [ ]:
## en X guardamos todas las listas de rasgos

X=[]
for language in D_lenguas.keys():
    X+=[list(D_lenguas[language].values())]

In [ ]:
## en X guardamos todas las listas de rasgos

X_and=[]
for language in D_andinas.keys():
    X_and+=[list(D_andinas[language].values())]

In [ ]:
from sklearn.manifold import TSNE

model = TSNE(n_components=2, perplexity=15, random_state=0, metric='precomputed',square_distances=True)
X = model.fit_transform(X)

In [ ]:
model = TSNE(n_components=2, perplexity=5, random_state=0, metric='precomputed',square_distances=True)
X_and = model.fit_transform(X_and)

In [ ]:
#from sklearn.decomposition import PCA
#transformer = PCA(n_components=3)
#X = transformer.fit_transform(X)

In [ ]:
x=list(zip(*list(X)))[0]
y=list(zip(*list(X)))[1]

In [ ]:
x_and=list(zip(*list(X_and)))[0]
y_and=list(zip(*list(X_and)))[1]

In [ ]:
dominio_number=4

import matplotlib.pyplot as plt

cmap = plt.get_cmap('RdBu')

fig, ax = plt.subplots(dpi=800)

for i in range(len(x)):
    ax.annotate(list(D.keys())[i], weight='demi', color='k', xy=(x[i],y[i]), fontsize=4, alpha=0.65)
#    ax.annotate(labels[i], weight='demi', color='r', xy=(x[i]-1.5,y[i]-1.5), fontsize=4, alpha=0.85)

ax.plot([x[i] for i in [list(D.keys()).index(language) for language in D.keys()]],[y[i] for i in [list(D.keys()).index(language) for language in D.keys()]],'o',color='gold',markersize=5,markeredgewidth=0.5,markeredgecolor='k',alpha=0.75,fillstyle='full',clip_on=True)

#ax.plot([x[i] for i in [list(L.keys()).index(language) for language in L.keys() if family[language] in ['Mayan','Otomanguean']]],[y[i] for i in [list(L.keys()).index(language) for language in L.keys() if family[language] in ['Mayan','Otomanguean']]],'X',color='m',markersize=6,markeredgewidth=0.5,markeredgecolor='k',alpha=0.75,fillstyle='full',clip_on=True,label='Maya+Otomangue')
#ax.plot([x[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='South America']],[y[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='South America']],'H',color='orange',markersize=5,markeredgewidth=0.5,markeredgecolor=None,alpha=0.85,fillstyle='full',clip_on=True,label='South America')
#ax.plot([x[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='North America']],[y[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='North America']],'o',color='lime',markersize=5,markeredgewidth=0.5,markeredgecolor=None,alpha=0.85,fillstyle='full',clip_on=True,label='North America')

plt.grid(False)
plt.title('Clustering for domain '+str(dominio_number),fontsize=10)
ax.set_yticks([])
ax.set_xticks([])
#plt.legend(loc='best',fontsize=7)
plt.ylabel(r'dimension 2',fontsize=10)
plt.xlabel(r'dimension 1',fontsize=10)
plt.rcParams.update({'font.size': 10})
plt.savefig('2D/PCA_'+str(dominio_number)+'.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
dominio_number=4

import matplotlib.pyplot as plt

cmap = plt.get_cmap('RdBu')

fig, ax = plt.subplots(dpi=800)

for i in range(len(x_and)):
    ax.annotate(list(D_andinas.keys())[i], weight='demi', color='k', xy=(x_and[i],y_and[i]), fontsize=4, alpha=0.65)
#    ax.annotate(labels[i], weight='demi', color='r', xy=(x[i]-1.5,y[i]-1.5), fontsize=4, alpha=0.85)

ax.plot([x_and[i] for i in [list(D_andinas.keys()).index(language) for language in D_andinas.keys()]],[y_and[i] for i in [list(D_andinas.keys()).index(language) for language in D_andinas.keys()]],'o',color='gold',markersize=5,markeredgewidth=0.5,markeredgecolor='k',alpha=0.75,fillstyle='full',clip_on=True)

#ax.plot([x[i] for i in [list(L.keys()).index(language) for language in L.keys() if family[language] in ['Mayan','Otomanguean']]],[y[i] for i in [list(L.keys()).index(language) for language in L.keys() if family[language] in ['Mayan','Otomanguean']]],'X',color='m',markersize=6,markeredgewidth=0.5,markeredgecolor='k',alpha=0.75,fillstyle='full',clip_on=True,label='Maya+Otomangue')
#ax.plot([x[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='South America']],[y[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='South America']],'H',color='orange',markersize=5,markeredgewidth=0.5,markeredgecolor=None,alpha=0.85,fillstyle='full',clip_on=True,label='South America')
#ax.plot([x[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='North America']],[y[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='North America']],'o',color='lime',markersize=5,markeredgewidth=0.5,markeredgecolor=None,alpha=0.85,fillstyle='full',clip_on=True,label='North America')

plt.grid(False)
plt.title('Clustering for domain '+str(dominio_number),fontsize=10)
ax.set_yticks([])
ax.set_xticks([])
#plt.legend(loc='best',fontsize=7)
plt.ylabel(r'dimension 2',fontsize=10)
plt.xlabel(r'dimension 1',fontsize=10)
plt.rcParams.update({'font.size': 10})
#plt.savefig('2D/PCA_'+str(dominio_number)+'.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
S = []
for n_clusters in range_n_clusters:
    
    cluster = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
    labels = cluster.labels_
    silhouette_avg = silhouette_score(X, labels)
    S.append(silhouette_avg)
    print("For n_clusters =", n_clusters,"The average silhouette_score is :", silhouette_avg)

In [ ]:
fig, ax = plt.subplots(dpi=800)

ax.plot(range_n_clusters, S, linewidth=0.5, marker='o',color='orange',markersize=5,markeredgecolor='k',markeredgewidth=1,fillstyle='full',clip_on=True)
plt.xlabel(r'number of clusters',fontsize=12)
plt.ylabel(r'silhoutte coefficient',fontsize=12)
plt.rcParams.update({'font.size': 10})
plt.savefig('2D/pca_evaluacion_custering.pdf', format='pdf', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
labels = kmeans.labels_

In [ ]:
labels

In [ ]:
labels_dict=dict(zip(list(D.keys()),labels))

In [ ]:
labels_dict

In [ ]:
x=list(zip(*list(X)))[0]
y=list(zip(*list(X)))[1]

In [ ]:
dominio_number=4

areas = pd.read_csv('areas.csv',sep=',')
lenguas = [name.lower() for name in list(areas['lengua'])]
areas_dict = {}
for i in list(areas.index):
    if areas.loc[i,'lengua'].lower()=='moseten':
        areas_dict['moseten']=[areas.loc[i,'sub-area'].lower(),areas.loc[i,'birchall-areas'].lower(),areas.loc[i,'birchall-east-west'].lower(),areas.loc[i,'area'].lower()]
    else:
        areas_dict[areas.loc[i,'lengua'].lower()]=[areas.loc[i,'sub-area'].lower(),areas.loc[i,'birchall-areas'].lower(),areas.loc[i,'birchall-east-west'].lower(),areas.loc[i,'area'].lower()]
    #nombres = list(D.columns)
import matplotlib.pyplot as plt

cmap = plt.get_cmap('RdBu')

fig, ax = plt.subplots(dpi=800)

hipotesis = 0

for i in range(len(x)):
    if hipotesis==0:
        if list(D.keys())[i]=='cavineã±a':
            arreglo='cavineña'
            ax.annotate(areas_dict[arreglo], color='k', xy=(x[i],y[i]), fontsize=5, alpha=0.65)
        else:
            ax.annotate(list(D.keys())[i], color='k', xy=(x[i],y[i]), fontsize=5, alpha=0.65)
    else:
        if list(D.keys())[i]=='cavineã±a':
            arreglo='cavineña'
            ax.annotate(areas_dict[arreglo][hipotesis-1], color='k', xy=(x[i],y[i]), fontsize=5, alpha=0.65)
        else:
            ax.annotate(areas_dict[list(D.keys())[i]][hipotesis-1], weight='demi', color='k', xy=(x[i],y[i]), fontsize=4, alpha=0.65)
    #ax.annotate(labels[i], weight='demi', color='r', xy=(x[i]-1.5,y[i]-1.5), fontsize=4, alpha=0.85)

ax.plot([x[i] for i in [list(D.keys()).index(language) for language in D.keys() if labels_dict[language]==0]],[y[i] for i in [list(D.keys()).index(language) for language in D.keys() if labels_dict[language]==0]],'o',color='gold',markersize=5,markeredgewidth=0.5,markeredgecolor='gold',alpha=0.75,fillstyle='full',clip_on=True,label='cluster 0')
ax.plot([x[i] for i in [list(D.keys()).index(language) for language in D.keys() if labels_dict[language]==1]],[y[i] for i in [list(D.keys()).index(language) for language in D.keys() if labels_dict[language]==1]],'o',color='cyan',markersize=5,markeredgewidth=0.5,markeredgecolor='cyan',alpha=0.75,fillstyle='full',clip_on=True,label='cluster 1')

ax.plot([x[i] for i in [list(D.keys()).index(language) for language in D.keys() if areas_dict[language][3].strip()=='andes']],[y[i] for i in [list(D.keys()).index(language) for language in D.keys() if areas_dict[language][3].strip()=='andes']],'x',color='k',markersize=7,markeredgewidth=0.5,markeredgecolor='k',alpha=0.75,fillstyle='full',clip_on=True,label='Andean languages')

#ax.plot([x[i] for i in [list(D.keys()).index(language) for language in D.keys() if labels_dict[language]==2]],[y[i] for i in [list(D.keys()).index(language) for language in D.keys() if labels_dict[language]==2]],'D',color='fuchsia',markersize=5,markeredgewidth=0.5,markeredgecolor='k',alpha=0.75,fillstyle='full',clip_on=True,label='cluster 2')

#ax.plot([x[i] for i in [list(L.keys()).index(language) for language in L.keys() if family[language] in ['Mayan','Otomanguean']]],[y[i] for i in [list(L.keys()).index(language) for language in L.keys() if family[language] in ['Mayan','Otomanguean']]],'X',color='m',markersize=6,markeredgewidth=0.5,markeredgecolor='k',alpha=0.75,fillstyle='full',clip_on=True,label='Maya+Otomangue')
#ax.plot([x[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='South America']],[y[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='South America']],'H',color='orange',markersize=5,markeredgewidth=0.5,markeredgecolor=None,alpha=0.85,fillstyle='full',clip_on=True,label='South America')
#ax.plot([x[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='North America']],[y[i] for i in [list(G.nodes()).index(language) for language in G.nodes() if language in id_family.keys() and macroarea[glotto_iso[language]]=='North America']],'o',color='lime',markersize=5,markeredgewidth=0.5,markeredgecolor=None,alpha=0.85,fillstyle='full',clip_on=True,label='North America')

plt.grid(False)
plt.title('Low-dimensional representation of languages of South America',fontsize=10)
ax.set_yticks([])
ax.set_xticks([])
plt.legend(loc='best',fontsize=7)
plt.ylabel(r'dimension 2',fontsize=10)
plt.xlabel(r'dimension 1',fontsize=10)
plt.rcParams.update({'font.size': 10})
plt.savefig('hipotesis/PCAclustering_4_'+str(hipotesis)+'.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

In [ ]:
world = world[world['continent']=='South America']

In [ ]:
x, y, codes = languoid['Longitude'].values, languoid['Latitude'].values, languoid['ID'].values

In [ ]:
codes = [glotto_lenguas[L] for L in codes]

In [ ]:
labels_dict.keys()

In [ ]:
len(x),len(D)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(dpi=800)

ax.set_aspect('equal')
plt.title('Geographic distribution of clusters',fontsize=7)
world.plot(ax=ax, color='white', edgecolor='salmon',linewidth=0.2)
#cities.plot(ax=ax, marker='o', color='red', markersize=5)
ax.plot([x[i] for i in [codes.index(language) for language in codes if language in labels_dict.keys() and labels_dict[language]==0]],[y[i] for i in [codes.index(language) for language in codes if language in labels_dict.keys() and labels_dict[language]==0]],'o',color='gold',markersize=5,markeredgewidth=0.5,markeredgecolor='gold',alpha=0.75,fillstyle='full',clip_on=True,label='cluster 0')
ax.plot([x[i] for i in [codes.index(language) for language in codes if language in labels_dict.keys() and labels_dict[language]==1]],[y[i] for i in [codes.index(language) for language in codes if language in labels_dict.keys() and labels_dict[language]==1]],'o',color='cyan',markersize=5,markeredgewidth=0.5,markeredgecolor='cyan',alpha=0.75,fillstyle='full',clip_on=True,label='cluster 1')

ax.plot([x[i] for i in [codes.index(language) for language in codes if areas_dict[language][3].strip()=='andes']],[y[i] for i in [codes.index(language) for language in codes if areas_dict[language][3].strip()=='andes']],'x',color='k',markersize=5,markeredgewidth=0.25,markeredgecolor='k',alpha=0.75,fillstyle='full',clip_on=True,label='Andean languages')

#for i in range(len(x)):
#        ax.annotate(codes[i], color='k', xy=(x[i],y[i]), fontsize=2, alpha=0.65)
        
plt.legend(loc='best',fontsize=5)
ax.set_yticks([])
ax.set_xticks([])
plt.savefig('maps/mapa_clusters.jpg', format='jpg', transparent=True, bbox_inches='tight',dpi=800)
plt.show()

In [ ]:
areas_dict

### Arealidad de los rasgos!

In [ ]:
## filter group_dict

group_dict = {n:[k for k in labels_dict.keys() if labels_dict[k] == n] for n in set(labels_dict.values())}

In [ ]:
group_dict

In [ ]:
## formato sklearn!

#X = [list(D[language].values()) for language in D.keys() if language in labels_dict.keys()]
y = [labels_dict[language] for language in D.keys() if language in labels_dict.keys()]

In [ ]:
X

In [ ]:
y

#### manera 1: trees!

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

forest = ExtraTreesClassifier(n_estimators=250,random_state=0)

In [ ]:
forest.fit(X, y)
importances = dict(zip(D[list(D.keys())[0]].keys(),forest.feature_importances_))

In [ ]:
## ordenamos los rasgos de mayor a menor importancia!

importances = {k: v for k, v in sorted(importances.items(), key=lambda item: item[1],reverse=True)}

In [ ]:
importances

#### manera 2: kbest!

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
kbest = SelectKBest(chi2, k='all').fit(X, y)

In [ ]:
importances = dict(zip(D[list(D.keys())[0]].keys(),kbest.scores_))

In [ ]:
## ordenamos los rasgos de mayor a menor importancia!

importances = {k: v for k, v in sorted(importances.items(), key=lambda item: item[1],reverse=True)}

In [ ]:
importances